In [10]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [22]:
stats_df = pd.read_csv('inputs/04_csv/Ecoregion_Stats_TerraClimate_SRTM.csv')
print(stats_df.shape)
print(stats_df)

(179, 19)
             system:index                                      BIOME_NAME  \
0    000000000000000000e4                                       Mangroves   
1    000000000000000000ee                      Deserts & Xeric Shrublands   
2    0000000000000000008a    Tropical & Subtropical Dry Broadleaf Forests   
3    00000000000000000019    Tropical & Subtropical Dry Broadleaf Forests   
4    0000000000000000011b    Tropical & Subtropical Dry Broadleaf Forests   
..                    ...                                             ...   
174  0002000000000000003f  Tropical & Subtropical Moist Broadleaf Forests   
175  00020000000000000041  Tropical & Subtropical Moist Broadleaf Forests   
176  0002000000000000004d  Tropical & Subtropical Moist Broadleaf Forests   
177  00020000000000000054  Tropical & Subtropical Moist Broadleaf Forests   
178  00020000000000000068  Tropical & Subtropical Moist Broadleaf Forests   

     BIOME_NUM    COLOR COLOR_BIO COLOR_NNH ECO_BIOME_  ECO_ID  \

In [23]:

stats_df = stats_df.drop(['.geo', 'system:index'], axis=1)
print(stats_df)

                                         BIOME_NAME  BIOME_NUM    COLOR  \
0                                         Mangroves       14.0  #E600AA   
1                        Deserts & Xeric Shrublands       13.0  #BFA244   
2      Tropical & Subtropical Dry Broadleaf Forests        2.0  #B5C435   
3      Tropical & Subtropical Dry Broadleaf Forests        2.0  #81B50A   
4      Tropical & Subtropical Dry Broadleaf Forests        2.0  #737400   
..                                              ...        ...      ...   
174  Tropical & Subtropical Moist Broadleaf Forests        1.0  #0D6101   
175  Tropical & Subtropical Moist Broadleaf Forests        1.0  #47B700   
176  Tropical & Subtropical Moist Broadleaf Forests        1.0  #0E5B00   
177  Tropical & Subtropical Moist Broadleaf Forests        1.0  #4DE600   
178  Tropical & Subtropical Moist Broadleaf Forests        1.0  #98E500   

    COLOR_BIO COLOR_NNH ECO_BIOME_  ECO_ID  \
0     #FE01C4   #7BC141       NO14     616   
1     #

In [24]:
stats_df.to_csv('outputs/csv/Ecoregion_Stats_TerraClimate_SRTM.csv', index=False)